# Análisis exploratorio de datos Formar Innovar

In [1]:
# Cargar librerías
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Leer las variables de entorno
MOODLE_URL = os.getenv("MOODLE_URL")
MOODLE_API_KEY = os.getenv("MOODLE_API_KEY")
MOODLE_FORMAT = "json"

In [3]:
def get_courses():
    params = {
        'wstoken': MOODLE_API_KEY,
        'wsfunction': 'core_course_get_courses',
        'moodlewsrestformat': MOODLE_FORMAT
    }
    response = requests.get(MOODLE_URL, params=params)
    return response.json()

In [4]:
# Función para obtener los usuarios matriculados en un curso
def get_enrolled_users(course_id):
    params = {
        'wstoken': MOODLE_API_KEY,
        'wsfunction': 'core_enrol_get_enrolled_users',
        'moodlewsrestformat': MOODLE_FORMAT,
        'courseid': course_id
    }
    response = requests.get(MOODLE_URL, params=params)
    return response.json()

In [5]:
# Función para obtener todos los usuarios
def get_all_users():
    params = {
        'wstoken': MOODLE_API_KEY,
        'wsfunction': 'core_user_get_users',
        'moodlewsrestformat': MOODLE_FORMAT,
        'criteria[0][key]': '',  # No especificamos ningún criterio para traer todos los usuarios
        'criteria[0][value]': ''
    }
    response = requests.get(MOODLE_URL, params=params)
    return response.json()

# Obtener todos los usuarios
users_data = get_all_users()

# Verificar si hay datos de usuarios
if 'users' not in users_data:
    print("No se encontraron usuarios o no tienes acceso a los datos de usuarios.")
else:
    # Convertir los usuarios a un DataFrame para facilitar el análisis
    users_df = pd.DataFrame(users_data['users'])

    # Filtrar usuarios con el rol de "Estudiante" en customfields
    def is_student(customfields):
        for field in customfields:
            if field['shortname'] == 'Rol' and field['value'] == 'Estudiante':
                return True
        return False

    # Aplicar el filtro
    students_df = users_df[users_df['customfields'].apply(is_student)]

    # Mostrar los estudiantes
    print(f"Se encontraron {len(students_df)} estudiantes:")
    print(students_df[['id', 'username', 'fullname']])

    # Si quieres guardar la información en un archivo CSV
    students_df.to_csv('estudiantes.csv', index=False)

Se encontraron 374 estudiantes:
       id      username                      fullname
0       1         guest                  Guest user  
47   1532  estudiante12             Estudiante 1111 1
48   1533   estudiante2                  Estudiante 2
49   1534   estudiante3                  estudiante 3
50   1535   estudiante4                  Estudiante 4
..    ...           ...                           ...
417  1904    1141123688   Maily Zamara Durán Martínez
420  1907    1031849817         Timoteo Castro Torres
421  1908    1070331986  Carlos Fernando Durán Méndez
422  1909    1014898800          Matías Sanchez Cabra
426  1914    1221717161      Ainhoa Sánchez Aranguren

[374 rows x 3 columns]


In [24]:
get_enrolled_users(207)

{'exception': 'webservice_access_exception',
 'errorcode': 'accessexception',
 'message': 'Excepción al control de acceso'}

In [8]:
# 1. Obtener la lista de cursos
courses_data = get_courses()
courses_df = pd.DataFrame(courses_data)

# 2. Iterar sobre cada curso y obtener los estudiantes matriculados
enrolled_users_by_course = {}

for index, course in courses_df.iterrows():
    course_id = course['id']
    course_name = course['fullname']
    
    # Obtener los estudiantes matriculados en este curso
    enrolled_users_data = get_enrolled_users(course_id)
    
    # Verificar si la respuesta contiene datos de usuarios matriculados
    if isinstance(enrolled_users_data, list) and enrolled_users_data:
        enrolled_users_df = pd.DataFrame(enrolled_users_data)
    else:
        enrolled_users_df = pd.DataFrame()  # Crear un DataFrame vacío si no hay estudiantes
    
    # Guardar el DataFrame en el diccionario con el ID del curso como clave
    enrolled_users_by_course[course_id] = enrolled_users_df
    
    # Mostrar los primeros registros de usuarios matriculados, si existen
    if not enrolled_users_df.empty:
        print(f"Usuarios matriculados en el curso '{course_name}' (ID {course_id}):")
        print(enrolled_users_df.head())
    else:
        print(f"No hay usuarios matriculados en el curso '{course_name}' (ID {course_id}).")
    
    print("\n")

Usuarios matriculados en el curso 'FORMAR INNOVAR' (ID 1):
   id  username        firstname       lastname  \
0   1     guest       Guest user                  
1   2     admin           System  Administrator   
2   6  carlosfc  Carlos Fernando  Cometa Hortúa   
3   9  35251059     Nubia Sheena    Achury Mora   
4  20  65800442     Diana Mireya  Tafur Sánchez   

                        fullname                                  email  \
0                   Guest user                           root@localhost   
1           System Administrator                admin@formarinnovar.com   
2  Carlos Fernando Cometa Hortúa                 carlosfcometa@yahoo.es   
3       Nubia Sheena Achury Mora  director@politecnicoformarinnovar.com   
4     Diana Mireya Tafur Sánchez           rectorfusa@formarinnovar.com   

     department  firstaccess  lastaccess  lastcourseaccess  ...  \
0                          0           0                 0  ...   
1                 1520258061  1696806004         

KeyboardInterrupt: 

In [10]:
def get_enrolled_users(course_id):
    params = {
        'wstoken': MOODLE_API_KEY,
        'wsfunction': 'core_enrol_get_enrolled_users',
        'moodlewsrestformat': MOODLE_FORMAT,
        'courseid': course_id
    }
    response = requests.get(MOODLE_URL, params=params)
    return response.json()

# Ejemplo: Obtener los usuarios matriculados en un curso con ID 2
enrolled_users_data = get_enrolled_users(2)
enrolled_users_df = pd.DataFrame(enrolled_users_data)
